In [17]:
import pandas as pd
from kinconnect_api.config import RAW_DATA_DIR, load_dotenv

load_dotenv()
# fake_profile_data = pd.read_csv(f"{RAW_DATA_DIR}/fake_profile_data.csv")
# fake_profile_data.head()
fake_profile_data = pd.read_csv(f"{RAW_DATA_DIR}/fake_profile_data_3.csv")
fake_profile_data.head()

,What is your name?,"What are your interests? (ie technical topic, coding language, business problem).","If you have a project idea, describe your idea . Please include whether what sector it is in, and what business problem it is solving and for whom. If you don’t have a project, skip this question.","What is your strongest functional role (such as developer, UX, business, product)? Please share one or two things about your experience in role your experience, for example a success, companies you worked for, how many years experience, a challenging project, etc.",Career path from Linkedin,Are you interested in meeting people with a specific skill set (either one that you lack or one that you already have but want to clone yourself to speed up building). What is the skills sets that you are looking to meet?,Past Projects Portfolio
0,Chloe,I'm interested in Rags and AI LLMs,NaN,"Back end developer, 10 years, build and deploy...",Senior Software EngineerSenior Software Engine...,"Product, frontend,",### Project 1: **Netflix Recommendation Engine...
1,Jiro,I'm interested in using AI to solve climate ch...,How can AI use to regulate office temperature ...,Product manager. I worked as a product manager...,Company: Uber\nTitle: Principal Program Manage...,"Front end developer, Back end developer",### Project 1: **Uber Driver App Redesign**\n\...
2,Abdul,I'm interested in using AI to make building co...,Using AI to make building construction and dev...,Business Strategy. Business Development. Marke...,They are a Business Development Manager with o...,"Front end developer, Back end developer",### Project 1: **Market Expansion Strategy**\n...
3,Tim,I'm interested in using AI to solve the loneli...,Match making,Frontend engineer. I worked at building the U...,\nThey are a Frontend Engineer with extensive ...,"Business strategy, Back end developer, product",### Project 1: **Real-Time Incident Reporting ...
4,Elijah,I'm interested in using AI to create more empathy,How can AI be used to make people think more a...,Backend engineer. I have worked on databases f...,Micro Quant Fund · Full-timeMicro Quant Fund ·...,Product managers. Strategic business. Maybe so...,### Project 1: **Slack Data Synchronization Sy...


In [18]:
profile_list = []

for index, row in fake_profile_data.iterrows():
    profile_dict = {'questions_and_answers': []}
    for column in fake_profile_data.columns:
        qa_pair = {"question": column, "answer": row[column]}
        profile_dict['questions_and_answers'].append(qa_pair)
    profile_list.append(profile_dict)

profile_details_list = []
for index, row in fake_profile_data.iterrows():
    profile_dict = {'questions_and_answers': []}
    counter = 0
    for column in fake_profile_data.columns:
        qa_pair = {"question": column, "answer": row[column]}
        profile_dict['questions_and_answers'].append(qa_pair)
        if counter >= 3:
            break
        counter += 1
    profile_details_list.append(profile_dict)

# Convert each profile's questions and answers into a formatted string and store in a list
formatted_profiles = [f'\n\n'.join(f"## {pair['question']}: \n {pair['answer']}" for pair in profile['questions_and_answers']) for profile in profile_list]
formatted_profile_details = [f'\n\n'.join(f"## {pair['question']}: \n {pair['answer']}" for pair in profile['questions_and_answers']) for profile in profile_details_list]
len(formatted_profiles)


11

In [19]:
formatted_profiles_df = pd.DataFrame(formatted_profiles, columns=["bio"])
formatted_profile_details_df = pd.DataFrame(formatted_profile_details, columns=["bio_details"])
formatted_profiles_df.to_csv(f"{RAW_DATA_DIR}/formatted_profiles.csv", index=False)
formatted_profile_details_df.to_csv(f"{RAW_DATA_DIR}/formatted_profile_details.csv", index=False)

In [ ]:
from pymongo import MongoClient
from pydantic import BaseModel, Field
from typing import List, Dict

class CareerEntry(BaseModel):
    company: str = Field(..., title="Company they worked at")
    title: str = Field(..., title="Title of the role they held")
    description: str = Field(..., title="Description of the role they held")
    start_date: str = Field(..., title="Start date of the role")
    end_date: str = Field(..., title="End date of the role")


class ProjectEntry(BaseModel):
    title: str = Field(..., title="Title of the project")
    description: str = Field(..., title="Description of the project")




In [ ]:

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chat_models.base import BaseChatModel
from textwrap import dedent
import logging
from pathlib import Path
import pandas as pd
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List, Dict, Callable, Union
import os
from tqdm import tqdm
from datetime import datetime
# Setup logging as per the new configuration

In [ ]:
career_extraction_prompt_str = dedent("""
You are a data generation app which creates profiles for participants at a hackathon. 
Given their form submission which asks them a few questions. Create a profile. 
Find all the relevant details about their career.

<form_submission>                                   
{submission}
</form_submission>
""")

In [ ]:
from langchain_fireworks import ChatFireworks
load_dotenv()

def get_career(profile):
    prompt_str = dedent("""
You are a data generation app which creates profiles for participants at a hackathon. 
Given their form submission which asks them a few questions. Create a profile. 
Find all the relevant details about their career.

<form_submission>                                   
{submission}
</form_submission>
""")
    mixtral_llm = ChatFireworks(
        model="accounts/fireworks/models/mistral-7b-instruct-v3")

    career_llm = mixtral_llm.with_structured_output(CareerEntry)

    prompt = ChatPromptTemplate.from_template(prompt_str)
    chain = prompt | career_llm
    return chain.invoke({"submission": profile})


career_profiles = [get_career(unstructured_profile) for unstructured_profile in formatted_profiles]


In [ ]:
career_profiles[0]


In [ ]:
from langchain_fireworks import ChatFireworks
load_dotenv()

def get_project(profile):
    prompt_str = dedent("""
You are a data generation app which creates profiles for participants at a hackathon. 
Given their form submission which asks them a few questions. Create a profile. 
Find all the relevant details about their past projects and portfolio.

<form_submission>                                   
{submission}
</form_submission>
""")
    mixtral_llm = ChatFireworks(
        model="accounts/fireworks/models/mistral-7b-instruct-v3")

    career_llm = mixtral_llm.with_structured_output(ProjectEntry)

    prompt = ChatPromptTemplate.from_template(prompt_str)
    chain = prompt | career_llm
    return chain.invoke({"submission": profile})


project_profiles = [get_project(unstructured_profile) for unstructured_profile in formatted_profiles]


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from textwrap import dedent
from langchain_core.pydantic_v1 import BaseModel, Field

load_dotenv()

def get_profile_details(profile):
    prompt_str = dedent("""
You are a data generation app which creates profiles for participants at a hackathon. 
Given their form submission which asks them a few questions. Create a profile. 
Find all the relevant details about their profile. Find their skills, accolades, honors and current interests.
Generate a name for the person. Make the name diverse and real representation of people working in the silicon valley with the relevant description of their form submission.
                        
<form_submission>                                   
{submission}
</form_submission>
""")
    # Define a Pydantic model for the profile
    class ProfileDetails(BaseModel):
        honors: List[str] = Field(None, title="Honors, Awards and recognition they have received in life")
        interests: List[str] = Field(..., title="Interests and current focus of theirs the work or the event")
        skills: List[str] = Field(..., title="Skills they have")
        name: str = Field(..., title="Name of the person")
        elevator_pitch: str = Field(..., title="Elevator pitch for the person for the event")
        
    gpt35_llm =  ChatOpenAI(model="gpt-3.5-turbo")

    career_llm = gpt35_llm.with_structured_output(ProfileDetails)

    prompt = ChatPromptTemplate.from_template(prompt_str)
    chain = prompt | career_llm
    return chain.invoke({"submission": profile})


profile_details = [get_profile_details(unstructured_profile) for unstructured_profile in formatted_profile_details]


In [ ]:
print(career_profiles[0])
print(project_profiles[0])
print(profile_details[0])

In [ ]:
from kinconnect_api.config import MONGO_CONNECTION_STRING
# Connect to MongoDB
client = MongoClient(MONGO_CONNECTION_STRING)
db = client['kinconnect']
profiles_collection = db['demo_profiles']

class CareerEntry(BaseModel):
    company: str
    title: str
    description: str
    start_date: str
    end_date: str

class ProjectEntry(BaseModel):
    title: str
    description: str

class ProfileModel(BaseModel):
    name: str = Field(..., title="Name of the person")
    honors: list[str] = Field(None, title="Honors, Awards and recognition they have recieved in life")
    interests: list[str] = Field(..., title="Interests and current focus of theirs the work or the event")
    skills: list[str] = Field(..., title="Skills they have")
    career: List[CareerEntry] = Field(..., title="Career history of the person")
    past_projects: List[ProjectEntry] = Field(..., title="Projects they have worked on")
    elevator_pitch: str = Field(..., title="Elevator pitch for the person for the event")

    class Config:
        arbitrary_types_allowed = True
zipped_profiles = zip(profile_details, career_profiles, project_profiles)


fake_profiles = [
    ProfileModel(
        name=profile_details.name,
        honors=profile_details.honors,
        interests=profile_details.interests,
        skills=profile_details.skills,
        career=[
            CareerEntry(
                company=career_profile['company'],
                title=career_profile['title'],
                description=career_profile['description'],
                start_date=career_profile['start_date'],
                end_date=career_profile['end_date']
            )
        ],
        past_projects=[
            ProjectEntry(
                title=project_profile['title'],
                description=project_profile['description']
            )
        ],
        elevator_pitch=profile_details.elevator_pitch
    )
    for profile_details, career_profile, project_profile in zipped_profiles
        
]

# Upsert fake profiles into MongoDB based on the name of the profile
for profile in fake_profiles:
    profiles_collection.update_one(
        {"name": profile.name},
        {"$set": profile.dict()},
        upsert=True
    )


In [ ]:
for profile in fake_profiles:
    print(profile)

In [ ]:

from fastapi import FastAPI, HTTPException
from pydantic import BaseModel, Field
from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain_fireworks import FireworksEmbeddings
from typing import List
from pymongo import MongoClient
from langchain_fireworks import ChatFireworks
from kinconnect_api.config import MONGO_CONNECTION_STRING, RAW_DATA_DIR, load_dotenv
import pandas as pd

def create_vector_search():
        """
        Creates a MongoDBAtlasVectorSearch object using the connection string, database, and collection names, along with the OpenAI embeddings and index configuration.

        :return: MongoDBAtlasVectorSearch object
        """
        vector_search = MongoDBAtlasVectorSearch.from_connection_string(
            MONGO_CONNECTION_STRING,
            "kinconnect.demo_profiles",
            FireworksEmbeddings(model="nomic-ai/nomic-embed-text-v1.5"),
            index_name="demo_embedding_idx2"
        )
        return vector_search


def perform_similarity_search(query, top_k=3):
    """
    This function performs a similarity search within a MongoDB Atlas collection. It leverages the capabilities of the MongoDB Atlas Search, which under the hood, may use the `$vectorSearch` operator, to find and return the top `k` documents that match the provided query semantically.

    :param query: The search query string.
    :param top_k: Number of top matches to return.
    :return: A list of the top `k` matching documents with their similarity scores.
    """
    vector_search = create_vector_search()

    results = vector_search.similarity_search_with_score(
        query=query,
        k=5,
    )

    return results

profiles = perform_similarity_search("I am looking to work with a backend engineer in a hackathon")

In [ ]:
print(profiles[0][0].page_content)
print(profiles[0][0].metadata['_id']['$oid'])


In [ ]:
profiles[0][0].metadata

In [6]:
import instructor
from typing import Optional
from enum import Enum
from pydantic import BaseModel, Field
from openai import OpenAI

class CompanyType(Enum):
    Startup = "<10 employees"
    SMB = "10-500 employees"
    Enterprise = ">500- employees"
    Other = "Other"

# Define your desired output structure
class UserBio(BaseModel):
    name: str = Field(..., title="What is the name of the person")
    email: str = Field(..., title="Email")
    company_name: str = Field(..., title="Company name")
    job_title: str = Field(..., title="Job title")
    job_description: str = Field(..., title="Job description")
    company_type: CompanyType = Field(..., title="How would you describe your company size?")
    hack_idea: Optional[str] = Field(..., title="What do you wanna build on/hack on?")
    def generate_bio_md(self):
        return f"## {self.name}\n- Age: {self.age}\n- Gender: {self.gender}\n- Location: {self.location}\n- Occupation: {self.occupation}"

# Patch the OpenAI client
client = instructor.from_openai(OpenAI())

# Extract structured data from natural language
user_info = client.chat.completions.create(
    model="gpt-3.5-turbo",
    response_model=UserBio,
    messages=[{"role": "user", "content": "Generate Realistic Bio of a person who lives in bay area."}],
)

print(user_info.generate_bio_md())
#> John Doe

#> 30

## Emily
- Age: 30
- Gender: female
- Location: Bay Area
- Occupation: Software Engineer


2024-06-18 14:36:22.073 | INFO     | kinconnect_api.config:<module>:15 - PROJ_ROOT path is: /Users/nehiljain/code/kinconnect/kinconnect_api


You can use the `os` module in Python to list all files in a directory. Here are a few ways to do it:

**Method 1: Using `os.listdir()`**
```
import os

dir_path = '/path/to/directory'
files = os.listdir(dir_path)
print(files)  # prints a list of all files and directories in the directory
```
**Method 2: Using `os.scandir()` (Python 3.5 and later)**
```
import os

dir_path = '/path/to/directory'
files = [entry.name for entry in os.scandir(dir_path) if entry.is_file()]
print(files)  # prints a list of all files in the directory
```
**Method 3: Using `glob` module**
```
import glob

dir_path = '/path/to/directory'
files = glob.glob(dir_path + '/*')
print(files)  # prints a list of all files in the directory
```
**Method 4: Using `pathlib` module (Python 3.4 and later)**
```
import pathlib

dir_path = pathlib.Path('/path/to/directory')
files = [file.name for file in dir_path.iterdir() if file.is_file()]
print(files)  # prints a list of all files in the directory
```
Note that these method